In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn import metrics
import matplotlib.pyplot as plt
import warnings

# Sopprime i warning
warnings.filterwarnings('ignore')

print("Avvio dell'analisi DBSCAN...")

# Carica il dataset
file_path = 'data/puliti/movimento_fe.csv'
try:
    df = pd.read_csv(file_path)

    # --- 1. Preparazione delle "Etichette Vere" (per la valutazione) ---
    df['Primary_Position'] = df['Position'].apply(lambda x: x.split(',')[0].strip() if pd.notnull(x) else 'Unknown')
    df = df[df['Primary_Position'] != 'Unknown']
    true_labels = df['Primary_Position']
    k_true = true_labels.nunique()
    print(f"Etichette vere ('Primary_Position') caricate. Ci sono {k_true} ruoli unici.")

    # --- 2. Preparazione delle "Features" (X) ---
    X = df.select_dtypes(include='number').copy()
    features_used = X.columns.tolist()
    print(f"Preprocessing di {len(features_used)} features numeriche...")
    
    # --- 3. Preprocessing (Imputazione e Scaling) ---
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imputed)
    print("Dati imputati e scalati pronti.")

    # --- 4. Analisi per trovare 'eps' e 'min_samples' ---
    D = X_scaled.shape[1]
    min_samples = 2 * D
    print(f"Dimensioni (features): {D}. Imposto min_samples = {min_samples}")

    print("Calcolo delle distanze dei vicini (NearestNeighbors)...")
    nn = NearestNeighbors(n_neighbors=min_samples, n_jobs=-1)
    nn.fit(X_scaled)
    distances, indices = nn.kneighbors(X_scaled)
    
    kth_distances = distances[:, min_samples - 1]
    kth_distances_sorted = np.sort(kth_distances)

    # --- 5. Trovare 'eps' (Metodo alternativo) ---
    # Dato che l'installazione di 'kneed' è fallita,
    # usiamo un'euristica comune: il 95° percentile.
    chosen_eps = np.percentile(kth_distances_sorted, 95)
    print(f"Valore 'eps' trovato (euristica 95° percentile): {chosen_eps:.4f}")

    # --- 6. Salvare il grafico del gomito ---
    plt.figure(figsize=(10, 6))
    plt.plot(kth_distances_sorted)
    plt.hlines(chosen_eps, xmin=0, xmax=len(kth_distances_sorted), linestyles='--', colors='r', label=f'eps (95° percentile)={chosen_eps:.4f}')
    plt.legend()
    plt.xlabel("Punti (ordinati per distanza)")
    plt.ylabel(f"Distanza dal {min_samples}-esimo vicino")
    plt.title("Analisi 'Elbow' per DBSCAN (k-distance plot)")
    plt.savefig('dbscan_eps_plot.png')
    print("Grafico 'dbscan_eps_plot.png' salvato.")

    # --- 7. Eseguire DBSCAN ---
    print(f"Esecuzione di DBSCAN in corso (eps={chosen_eps:.4f}, min_samples={min_samples})...")
    dbscan = DBSCAN(eps=chosen_eps, min_samples=min_samples, n_jobs=-1)
    cluster_labels = dbscan.fit_predict(X_scaled)

    # --- 8. Analisi Risultati DBSCAN ---
    n_clusters_ = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
    n_noise_ = list(cluster_labels).count(-1)
    
    print(f"\n--- Risultati DBSCAN ---")
    print(f"Numero di cluster trovati (escluso rumore): {n_clusters_}")
    print(f"Numero di punti 'rumore' (etichetta -1): {n_noise_} (su {len(df)} totali)")

    # --- 9. Valutazione e Confronto ---
    ari_score = metrics.adjusted_rand_score(true_labels, cluster_labels)
    nmi_score = metrics.normalized_mutual_info_score(true_labels, cluster_labels)
    
    print("\n--- Metriche di Valutazione (vs Etichette Reali) ---")
    print(f"Adjusted Rand Index (ARI): {ari_score:.4f}")
    print(f"Normalized Mutual Information (NMI): {nmi_score:.4f}")

    print("\n--- Confronto con KMeans (k=11, tutte le features) ---")
    print("KMeans ARI (precedente): 0.2491")
    print("KMeans NMI (precedente): 0.3697")
    
    if ari_score > 0.2491:
        print("Risultato: DBSCAN è più allineato ai ruoli.")
    else:
        print("Risultato: KMeans (forzato a 11 cluster) era più allineato ai 11 ruoli.")

    # --- 10. Matrice di Contingenza ---
    df['DBSCAN_Cluster'] = cluster_labels
    contingency_matrix_dbscan = pd.crosstab(df['Primary_Position'], df['DBSCAN_Cluster'])
    print("\n--- Matrice di Contingenza DBSCAN (Righe: Posizioni Vere, Colonne: Cluster Trovati) ---")
    print("La colonna '-1' rappresenta il 'rumore' (outlier)")
    print(contingency_matrix_dbscan)
    contingency_matrix_dbscan.to_csv('contingency_matrix_dbscan.csv')

except FileNotFoundError:
    print(f"Errore: File '{file_path}' non trovato.")
except Exception as e:
    print(f"Si è verificato un errore: {e}")